In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os

# Add the parent directory to the path
sys.path.append(os.path.dirname(os.getcwd()))

from infodump_tools.calculate import load_dfs, get_dfs_for_site
import polars as pl
from polars import col

In [ ]:
(
    joinyears,
    df_users,
    df_posts_all,
    df_comments_all,
    df_activity_all,
) = load_dfs("../infodump")

In [ ]:
(
    df_posts,
    df_comments,
    df_activity,
    df_months,
) = get_dfs_for_site("askme", df_posts_all, df_comments_all, df_activity_all)

In [ ]:
df_best = (
    df_months.join(
        df_posts.select("month", "postid")
        .join(
            df_comments.select("postid", "best"),
            on="postid",
            how="left",  # left join, so we keep posts with no comments
            coalesce=True,
        )
        .group_by("postid")
        .agg(
            month=col("month").first(),
            best_sum=col("best").eq(1).sum(),
            best_any=col("best").eq(1).any(),
        ),
        on="month",
        how="left",
    )
    .group_by("month")
    .agg(
        bests=col("best_sum").sum(),
        posts_with_best=col("best_any").sum(),
    )
)

df_best